<a href="https://colab.research.google.com/github/paulynamagana/AFDB_notebooks/blob/main/AFDB_MSA_Coverage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analysing AlphaFold's Database Multiple Sequence Alignments (MSAs) for coverage

This notebook provides a suite of tools to analyse the Multiple Sequence Alignment (MSA) associated with a predicted structure from the AlphaFold Database (AFDB). By dissecting the input data, we can better assess the reliability of the output structure.

**Background:**
The accuracy of AlphaFold is heavily dependent on the quality of its input MSA. The model uses these alignments to detect **co-evolutionary signals**, correlated mutations between pairs of residues that suggest physical proximity in 3D space.
* **Deep, Diverse MSAs** provide rich co-evolutionary constraints, leading to high-confidence structures.
* **Shallow or Biased MSAs** force the model to rely on single-sequence physics, often resulting in lower confidence (pLDDT) or "hallucinated" secondary structures.



---



- **What is an MSA?** A Multiple Sequence Alignment (MSA) is a collection of evolutionarily related protein sequences that have been aligned to highlight conserved positions and evolutionary changes. For AlphaFold, the MSA is the primary input, second only to the query sequence itself.



In [ ]:
#@title #1.&nbsp; Import and install libraries
!pip install -q py3Dmol biopython

import json
import requests
import re
import py3Dmol
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from IPython.display import display, Image
from io import StringIO
from Bio import AlignIO

print("----All packages imported successfully---")

##2.&nbsp; Fetch prediction summary from AFDB API

First, we query the AFDB API for our target UniProt ID to get a summary of all available models, including isoforms.

In [ ]:
uniprot_id =  "Q5VSL9" #@param {type:"string"}

url_pdb_prediction = f"https://alphafold.ebi.ac.uk/api/prediction/{uniprot_id.strip()}"

try:
    response = requests.get(url_pdb_prediction, timeout=20)
    if response.status_code == 200:
        result = response.json()
        print(json.dumps(result, indent=2))
    else:
        print(f"Error getting prediction for {uniprot_id}")
except requests.RequestException as e:
    print(f"Error during request: {e}")

In [ ]:
#@markdown This code block will print a table of all available entries for that UniProt ID, which can include isoforms.

# count how many isoforms
import pandas as pd

print(f"All entries for the UniProt ID {uniprot_id}:")

rows= []

for item in result:
    modelID = item.get("modelEntityId")
    description = item.get("uniprotDescription")
    latestVersion = item.get("latestVersion")
    average_plddt = item.get("globalMetricValue")
    fractionPlddtVeryLow = item.get("fractionPlddtVeryLow")
    fractionPlddtLow = item.get("fractionPlddtLow")
    fractionPlddtConfident = item.get("fractionPlddtConfident")
    fractionPlddtVeryHigh = item.get("fractionPlddtVeryHigh")

    rows.append({
        "model_ID": modelID,
        "description":description,
        "average_plddt": average_plddt,
        "latest_version": latestVersion,
        "fractionPlddtVeryLow": "{:.1f}%".format(fractionPlddtVeryLow * 100),
        "fractionPlddtLow": "{:.1f}%".format(fractionPlddtLow* 100),
        "fractionPlddtConfident": "{:.1f}%".format(fractionPlddtConfident* 100),
        "fractionPlddtVeryHigh": "{:.1f}%".format(fractionPlddtVeryHigh* 100)
    })

df = pd.DataFrame(rows)
df



In [ ]:
#@markdown Now, choose one model ID to analyse the MSA.
#@markdown This code block will first visualise the structure and the PAE plot
model = "AF-Q5VSL9-F1"#@param {type:"string"}

for entry in result:
    if entry["modelEntityId"] == model:
        plddt_url = entry["plddtDocUrl"]
        msa_url = entry["msaUrl"]
        pdbUrl = entry["pdbUrl"]
        paeDocUrl = entry["paeDocUrl"]
        break
if not model:
    print(f"Model {model} not found in results.")


def plot_pae(pae_url):
    #print(f"Fetching PAE data from: {pae_url}")
    response = requests.get(pae_url)
    data = response.json()

    if isinstance(data, list) and len(data) > 0:
        pae_matrix = data[0]['predicted_aligned_error']
    else:
        print("Invalid PAE data format.")
        return

    # Plotting
    plt.figure(figsize=(6, 6))
    plt.imshow(pae_matrix, cmap='Greens_r', vmin=0, vmax=31)

    cb = plt.colorbar()
    cb.set_label('Expected Position Error (Å)')
    plt.title(f'Predicted Aligned Error (PAE)\nModel: {model}')
    plt.xlabel('Scored Residue')
    plt.ylabel('Aligned Residue')
    plt.grid(False)
    plt.show()

# Function to show protein structure and image
def show_structure_and_image(pdb_url, paeUrl, color="plDDT"):
    # Retrieve the PDB data from the URL
    pdb_data = requests.get(pdb_url).text
    # Create a 3Dmol.js view
    view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')
    # Add the PDB data to the view
    view.addModel(pdb_data, 'pdb')

    if color == "plDDT":
        view.setStyle({'cartoon': {'colorscheme': {'prop': 'b', 'gradient': 'roygb', 'min': 50, 'max': 90}}})
    elif color == "rainbow":
        view.setStyle({'cartoon': {'color': 'spectrum'}})

    # Zoom to the structure
    view.zoomTo()
    display(view) # Display the 3D structure
    plot_pae(paeUrl) # Display PAE plot

show_structure_and_image(pdbUrl, paeDocUrl, "plDDT")


In [ ]:
#@markdown This code block will parse the MSA. It also prints the number of sequences found.

if plddt_url and msa_url:
    #print(f"--Fetching pLDDT from: {plddt_url}")
    plddt_response = requests.get(plddt_url)
    plddt_data = plddt_response.json()
    #print(plddt_data)

    plddt_scores = plddt_data['confidenceScore']
    residue_number = plddt_data['residueNumber']
    #print(f"Entry length: {len(plddt_scores)}")
    print(f"Successfully fetched pLDDT for {len(plddt_scores)} residues.")

    #print(f"--Fetching MSA from: {msa_url}")
    msa_response = requests.get(msa_url)
    msa_content = msa_response.text # Get the raw text of the .a3m file
    print("Successfully fetched MSA content.")

else:
    print(f"Error: Could not find URLs for model {model}")


def parse_a3m_content(a3m_content):
    """Parses the A3M content into a list of uppercase sequences."""
    sanitised_content = re.sub(r'[a-z]', '', msa_content)

    alignment = AlignIO.read(StringIO(sanitised_content), "fasta")

    #convert to a simple list of strings
    sequences = [str(record.seq) for record in alignment]

    return sequences

print("Parsing A3M content...")
sequences = parse_a3m_content(msa_content)
query_seq = sequences[0]
print(f"Parsing complete. Found {len(sequences)} sequences.")

## <font color='#2895ef'> MSA depth analysis

MSA Depth (or "Gap Coverage") is the count of sequences in the alignment that possess a non-gap residue at a specific position. It acts as a proxy for the amount of evolutionary information available for that specific part of the protein.

In [ ]:
#@markdown This code block will plot the MSA depth for the model entry ID given before

def get_msa_depth_from_content(sequences):
    """Calculates per-residue MSA depth from A3M file content."""

    seq_array = np.array([list(s) for s in sequences])

    non_gaps = (seq_array != '-') & (seq_array != '.')

    depths = np.sum(non_gaps, axis=0)
    return depths


# Calculate the MSA depths
msa_depths = get_msa_depth_from_content(sequences)

# Plotting
sequence_length = len(msa_depths)
residue_numbers = range(1, sequence_length + 1)
plt.figure(figsize=(12, 6))
plt.plot(residue_numbers, msa_depths)
plt.title(f"MSA Depth vs. Residue Number for {model}")
plt.xlabel("Residue Number (Sequence Position)")
plt.ylabel("MSA Depth (Number of non-gap sequences)")
plt.grid(True, linestyle='--', alpha=0.6)
plt.xlim(1, sequence_length)
plt.show()



**Interpretation guide:**

* **What is MSA Depth?** MSA depth is a per-residue count of the number of non-gap sequences in the alignment. It's a measure of how much information AlphaFold had for each specific position in the protein.

* **What is a good MSA?** A "good" MSA is both deep and diverse. It has many sequences but not too identical.And the sequences span a wide range of identities (e.g., from 30% to 90% identity to the query).

If the average depth across the protein is < 100 (or even < 30, as is common for orphans), it's a "shallow" MSA. This means AlphaFold has very little coevolutionary information. It will have to rely almost entirely on sequence features and its "knowledge" of protein physics.


A depth of 0 means only the query sequence has residues here; every other homolog in the MSA has a gap. This is the classic signature of a query-specific insertion or a low-complexity/disordered region that is not part of the conserved protein family domain.

## <font color='#2895ef'> Correlation: MSA depth vs. pLDDT

AlphaFold's confidence metric, **pLDDT** (predicted Local Distance Difference Test), estimates the local accuracy of the structure. We expect a strong correlation between Depth and pLDDT.

This first plot shows the MSA depth against a background coloured by the pLDDT score.

In [ ]:
#@markdown This code block will combine the pLDDT and the MSA depth in one plot

def plot_msa_depth_with_vertical_plddt_background(msa_depths, plddt_values, model_name, figsize=(14, 6)):
    """
    Plot MSA depth with vertical pLDDT-based background coloring
    Each residue position gets a vertical color based on its pLDDT value
    """

    sequence_length = len(msa_depths)
    residue_numbers = range(1, sequence_length + 1)

    fig, ax = plt.subplots(figsize=figsize)

    # Create vertical background colors based on pLDDT values
    for i, plddt in enumerate(plddt_values):
        if plddt > 90:
            color = "#024fcc"  # Very high - Dark Blue
            alpha = 0.5
        elif plddt > 70:
            color = "#60c2e8"  # Confident - Light Blue
            alpha = 0.5
        elif plddt > 50:
            color = "#f9d613"  # Low - Yellow
            alpha = 0.5
        else:
            color = "#FF7D45"  # Very low - Orange
            alpha = 0.5

        ax.add_patch(Rectangle((i + 0.5, 0), 1.02, max(msa_depths) * 1.1,
                      color=color, alpha=alpha, linewidth=0, zorder=0))

    ax.plot(residue_numbers, msa_depths, linewidth=2.0, color='#2c3e50',
            label='MSA Depth', marker='', zorder=5)

    ax.set_title(f"MSA Depth vs. Residue Number for {model_name}", fontsize=14, pad=20)
    ax.set_xlabel("Residue Number (Sequence Position)", fontsize=12)
    ax.set_ylabel("MSA Depth (Number of non-gap sequences)", fontsize=12)
    ax.grid(False)

    ax.set_xlim(0.5, sequence_length + 0.5)
    ax.set_ylim(0, max(msa_depths) * 1.1)

    # Create custom legend pLDDT
    from matplotlib.patches import Patch
    legend_elements = [
        Patch(facecolor='#024fcc', alpha=0.5, label='Very high (pLDDT > 90)'),
        Patch(facecolor='#60c2e8', alpha=0.5, label='Confident (70 < pLDDT ≤ 90)'),
        Patch(facecolor='#f9d613', alpha=0.5, label='Low (50 < pLDDT ≤ 70)'),
        Patch(facecolor='#FF7D45', alpha=0.5, label='Very low (pLDDT ≤ 50)'),
        plt.Line2D([0], [0], color='#2c3e50', lw=2, label='MSA Depth')
    ]

    ax.legend(handles=legend_elements, loc="upper left", bbox_to_anchor=(1.02, 1), borderaxespad=0.)

    plt.tight_layout()
    return fig, ax

fig2, ax2 = plot_msa_depth_with_vertical_plddt_background(msa_depths, plddt_scores, model)

plt.show()

#### Alternative plot (Twin-axis)

In [ ]:
#@markdown This plot shows the same data but with two separate y-axes (a "twin-axis" plot).

def plot_msa_depth_with_plddt_background(msa_depths, plddt_values, model_name, figsize=(14, 6)):
    """
    Plot MSA depth with pLDDT confidence regions as background colors

    Parameters:
    - msa_depths: list of MSA depths for each residue position
    - plddt_values: list of pLDDT values for each residue position
    - model_name: name of the model for the title
    - figsize: tuple for figure size
    """

    sequence_length = len(msa_depths)
    residue_numbers = range(1, sequence_length + 1)

    # Create the plot figure and axes
    fig, ax = plt.subplots(figsize=figsize)

    # Plot MSA depth as a line
    line = ax.plot(residue_numbers, msa_depths, linewidth=2, color='#2c3e50',
                   label='MSA Depth', marker='', markersize=1)

    # Customize the plot
    ax.set_title(f"MSA Depth vs. pLDDT for {model_name}", fontsize=14, pad=20)
    ax.set_xlabel("Residue number (sequence position)", fontsize=12)
    ax.set_ylabel("MSA depth (Number of non-gap sequences)", fontsize=12)

    # Set axis limits
    ax.set_xlim(1, sequence_length)
    ax.set_ylim(0, max(msa_depths) * 1.1)  # Add 10% padding at top

    # Add legend
    ax.legend(loc='upper right', framealpha=0.9)

    # Optional: Add secondary y-axis for pLDDT values
    ax2 = ax.twinx()
    ax2.plot(residue_numbers, plddt_values, color='red', alpha=1.0, linewidth=1.5,
             label='pLDDT')
    ax2.set_ylabel('pLDDT score', fontsize=12, color='red')
    ax2.set_ylim(0, 100)
    ax2.tick_params(axis='y', labelcolor='red')

    # Combine legends
    lines1, labels1 = ax.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax.legend(lines1 + lines2, labels1 + labels2, loc="upper left", bbox_to_anchor=(1.05, 1), borderaxespad=0.)
    plt.tight_layout()
    return fig, ax

fig1, ax1 = plot_msa_depth_with_plddt_background(msa_depths, plddt_scores, model)
plt.show()

## <font color='#2895ef'> Sequence coverage landscape

This raster plot provides a "bird's-eye view" of the alignment quality.
* **X-axis:** Residue position in the protein.
* **Y-axis:** Individual sequences in the MSA, sorted by identity to the query.
* **Color:** Sequence identity (Red = High identity, Blue = Low identity, White = Gap).


In [ ]:
#@markdown This code block will plot a raster plot (i.e. looks like the plot you get from ColabFold)

def create_msa_plot_matrix(sequences,):
    """
    Parses A3M content and creates a matrix for plotting.
    - Rows are sequences, sorted by identity to query.
    - Columns are residue positions.
    - Cell value is the sequence identity if not a gap, otherwise np.nan.
    """
    seq_array = np.array([list(s) for s in sequences])

    query_row = seq_array[0:1, :]
    matches = (seq_array == query_row)

    seq_identities = np.sum(matches, axis=1) / seq_array.shape[1]

    sorted_indices = np.argsort(seq_identities)
    sorted_identities = seq_identities[sorted_indices]
    sorted_seq_array = seq_array[sorted_indices]

    # Create the 2D matrix for imshow
    plot_matrix = np.tile(sorted_identities[:, np.newaxis], (1, seq_array.shape[1]))

    is_gap = (sorted_seq_array == '-') | (sorted_seq_array == '.')
    plot_matrix[is_gap] = np.nan

    return plot_matrix, sorted_identities

if 'sequences' in locals() and 'msa_depths' in locals():
    print("Parsing MSA and creating plot matrix...")
    plot_matrix, sorted_indices = create_msa_plot_matrix(sequences)
    num_hits, seq_length = plot_matrix.shape
    print(f"Matrix created with shape: {plot_matrix.shape}")

    plt.figure(figsize=(15, 8))
    ax = plt.gca()

    # ---Sequence Coverage Raster plot ---
    cmap = plt.get_cmap('jet')
    cmap.set_bad(color='white')

    im = ax.imshow(
        plot_matrix,
        aspect='auto',
        origin='lower',
        cmap=cmap,
        interpolation='nearest',
        extent=[1, seq_length, 0, num_hits])

    ax.plot(msa_depths, color='black', linewidth=1.5)

    # --- Colorbar and Labels ---
    cbar = plt.colorbar(im, ax=ax, pad=0.1)
    cbar.set_label("Sequence identity to query")
    ax.set_xlabel("Positions")
    ax.set_ylabel("Sequences")
    ax.set_xlim(1, seq_length + 1)
    ax.set_ylim(0, num_hits)
    plt.title("Sequence coverage")
    plt.tight_layout()
    plt.savefig("msa_coverage_plot.png")
    plt.show()

else:
    print("Error: Please run previous cells to get 'sequences' and 'msa_depths'.")


**What to look for:**
* **"Rainbow" Distribution:** A healthy MSA contains sequences spanning a broad range of identities (30% to 90%). This diversity drives the detection of co-evolution.
* **"Deserts" (White Regions):** Vertical bands of white indicate regions where most homologs have deletions. These often correspond to flexible loops or organism-specific insertions.